In [1]:
import os
import shutil
import pandas as pd
from zipfile import ZipFile
from datetime import datetime, timedelta
import pyodbc

pd.io.formats.excel.ExcelFormatter.header_style = None

In [2]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

conn2 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

In [3]:
qfechas = 'SELECT TmpAnioSemanaGenomma Año, TmpSemanaAnioGenomma Semana, TmpFecha FROM Gnm_DWH.dbo.Dim_Tiempo WHERE TmpDiaSemana = 7 AND TmpAnioSemanaGenomma > 2017;'
dfechas = pd.read_sql(qfechas, conn1)

qsemanas = 'SELECT SemID IDsemana, SemAnio Año, SemNumero Semana  FROM Gnm_MasterOp.dbo.CatSemanas WHERE SemAnio > 2017'
dsemanas = pd.read_sql(qsemanas, conn2)

c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Organization

Creación de la carpeta a la semana correspondiente:

In [4]:
stock_file = 'Genomma Lab_Walgreens_MULTIPERIOD_1965-10081294_20220315.xlsx'
other_file = 'Genomma Lab_Walgreens_MULTIPERIOD_1965-10081293_20220315.xlsx'
bty = 'Genomma Lab_Walgreens_EXPORT_1965-10081289_20220315.csv'
otc = 'Genomma Lab_Walgreens_EXPORT_1965-10081290_20220315.csv'
today = datetime.today() - timedelta(days=3)
year = datetime.today().year
week = datetime.today().isocalendar()[1]
path = r"C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\EUA\Walgreens\Ryans Files\{0}".format(year)
new_folder = path + '\\W {1} {0}'.format(year, week)

In [5]:
if not os.path.exists(new_folder):
    os.mkdir(new_folder)
bty_file = 'BTY {1} {0}.xlsx' 
otc_file = 'OTC {1} {0}.xlsx'

Creación de la carpeta Beauty y organización de los datos:

In [6]:
beauty_folder = new_folder + '\\BTY-OTC'
if not os.path.exists(beauty_folder):
    os.mkdir(beauty_folder)

zip_files = [file for file in os.listdir(path) if file.endswith('.zip')]
for zipfile in zip_files:
    with ZipFile(path + '\\' + zipfile, 'r') as zip_ref:
        zip_ref.extractall(beauty_folder)

Creación de la carpeta DC y organización de los datos:

Verificar si el nombre de cada archivo siempre es el mismo:

In [7]:
dc_folder = new_folder + '\\DC'
if not os.path.exists(dc_folder):
    os.mkdir(dc_folder)

shutil.move(path + '\\' + stock_file, dc_folder + '\\' + stock_file)
shutil.move(path + '\\' + other_file, new_folder + '\\' + other_file)

'C:\\Users\\jshernandezm\\OneDrive - genommalabinternacional\\02Paises\\EUA\\Walgreens\\Ryans Files\\2022\\W 11 2022\\Genomma Lab_Walgreens_MULTIPERIOD_1965-10081293_20220315.xlsx'

## Movimiento de los archivos semana anterior

In [8]:
curr_idweek = dsemanas.loc[(dsemanas['Año']==year)&(dsemanas['Semana']==week), 'IDsemana'].values[0]
last_idweek = curr_idweek - 1

ylw = dsemanas['Año'][dsemanas['IDsemana']==last_idweek].values[0] 
lw = dsemanas['Semana'][dsemanas['IDsemana']==last_idweek].values[0]

ylw = str(ylw).zfill(2)
lw = str(lw).zfill(2)

In [9]:
last_folder = r"C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\EUA\Walgreens\Ryans Files\{0}\W {1} {0}"
last_folder = last_folder.format(int(ylw), int(lw))
original_folder = last_folder + '\\ORIGINALS'
if not os.path.exists(original_folder):
    os.mkdir(original_folder)

shutil.move(last_folder + '\\BTY-OTC\\' + bty_file.format(ylw, lw), original_folder + '\\' + bty_file.format(ylw, lw))
shutil.move(last_folder + '\\BTY-OTC\\' + otc_file.format(ylw, lw), original_folder + '\\' + otc_file.format(ylw, lw))

'C:\\Users\\jshernandezm\\OneDrive - genommalabinternacional\\02Paises\\EUA\\Walgreens\\Ryans Files\\2022\\W 10 2022\\ORIGINALS\\OTC 10 2022.xlsx'

# Data Cleaning

In [10]:
dtypes = {'PLN NBR':'str',
          'UPC':'str',
          'ZIP':'str',
          'STORE':'str',
          'POG_IND':'str'}

## OTC

In [11]:
beauty_folder

'C:\\Users\\jshernandezm\\OneDrive - genommalabinternacional\\02Paises\\EUA\\Walgreens\\Ryans Files\\2022\\W 11 2022\\BTY-OTC'

In [12]:
%%time
df = pd.read_csv(beauty_folder + '\\' + otc, dtype=dtypes)

Wall time: 4.26 s


In [13]:
dates = [datetime.strptime(d, '%Y-%m-%d') for d in df['BEGIN_DATE'].unique()]
dates.sort()
dates = [d.strftime('%Y-%m-%d') for d in dates]

date_lw = dates[0]
date_cw = dates[1]

In [14]:
date_lw

'2022-02-27'

### Exportación de la semana anterior

In [15]:
%%time
df[df['BEGIN_DATE'] == date_lw].to_excel(last_folder + '\\BTY-OTC\\' + otc_file.format(ylw, str(lw).zfill(2)), index=False, sheet_name='Data Export')

Wall time: 3min 21s


### Exportación de la semana target

In [16]:
%%time
df[df['BEGIN_DATE'] == date_cw].to_excel(beauty_folder + '\\' + otc_file.format(year, str(week).zfill(2)), index=False, sheet_name='Data Export')

Wall time: 3min 11s


## Beauty

In [17]:
%%time
df = pd.read_csv(beauty_folder + '\\' + bty, dtype=dtypes)

Wall time: 4.92 s


### Exportación de la semana anterior

In [18]:
%%time
df[df['BEGIN_DATE'] == date_lw].to_excel(last_folder + '\\BTY-OTC\\' + bty_file.format(ylw, str(lw).zfill(2)), index=False, sheet_name='Data Export')

Wall time: 2min 51s


### Exportación de la semana target

In [19]:
%%time
df[df['BEGIN_DATE'] == date_cw].to_excel(beauty_folder + '\\' + bty_file.format(year, str(week).zfill(2)), index=False, sheet_name='Data Export')

Wall time: 2min 39s


No hay nada programado para cuando es cambio de año, o sea S1 2022 y S52 2021

In [20]:
os.rename(dc_folder + '\\' + stock_file, dc_folder + '\\DC.xlsx')

In [21]:
for csv_file in [f for f in os.listdir(beauty_folder) if f.endswith('.csv')]:
    os.remove(beauty_folder + '\\' + csv_file)

# Transform Files

## SO

### Import Data

In [29]:
year = 2022
week = 11

path_export = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\EUA\Walgreens\Output\{0}'
path = r"C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\EUA\Walgreens\Ryans Files\{0}".format(year)

new_folder = path + '\\W {1} {0}'.format(year, week)

In [30]:
%%time
df = pd.DataFrame()
bty_otc_files = new_folder + '\\BTY-OTC'
for file in os.listdir(bty_otc_files):
    aux = pd.read_excel(bty_otc_files + '\\' + file, dtype={'ZIP':'str'})
    df = pd.concat([df, aux])

Wall time: 4min 17s


### Data Cleaning

In [31]:
df[['Total Sales Amount', 'Total Sales Volume Units', 'Store On Hand Volume Units']].fillna(0, inplace=True)

data = df.pivot_table(index=['UPC', 'ITEM DESCRIPTION', 'ADDRESS1', 'CITY', 'STATE', 'ZIP', 'STORE'], 
                      values=['Total Sales Amount', 'Total Sales Volume Units', 'Store On Hand Volume Units'], 
                      aggfunc='sum').reset_index()

C:\Users\JSHERN~1\AppData\Local\Temp/ipykernel_18220/3502016242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Total Sales Amount', 'Total Sales Volume Units', 'Store On Hand Volume Units']].fillna(0, inplace=True)


In [32]:
data['Total Sales Volume Units'] = data['Total Sales Volume Units'].map(int)
data['Store On Hand Volume Units'] = data['Store On Hand Volume Units'].map(int)

In [33]:
data[['Total Sales Amount', 'Total Sales Volume Units', 'Store On Hand Volume Units']].sum()

Total Sales Amount             557825.46
Total Sales Volume Units        52906.00
Store On Hand Volume Units    1053001.00
dtype: float64

### Export Data

In [34]:
%%time
file_export = '{0}_{1}_Walgreens_SO.xlsx'
file_export = file_export.format(year, str(week).zfill(2))
cols = ['UPC', 'ITEM DESCRIPTION', 'ADDRESS1', 'CITY', 'STATE', 'ZIP', 'STORE', 'Total Sales Amount', 'Total Sales Volume Units', 'Store On Hand Volume Units']
data[cols].to_excel(path_export.format(year) + '\\' + file_export, index=False, sheet_name='Hoja1')

Wall time: 4min 19s


## Stock

In [35]:
weeks = [week]
for week in weeks:
    df = pd.read_excel(new_folder + '\\DC\\DC.xlsx', header=20, usecols='B:F')
    df.drop(0,axis=0,inplace=True)
    df_final = df[['UPC','ITEM DESCRIPTION','DC On Hand Volume Units']].copy()
    df_final['AÑO'] = year
    df_final['SEMANA'] = week
    df_final['UPC'] = df_final['UPC'].astype('int64') 

    df_final = df_final[['AÑO', 'SEMANA', 'UPC', 'ITEM DESCRIPTION', 'DC On Hand Volume Units']].copy()

    df_final.to_excel(path_export.format(year) + '\\{0}_{1}_Walgreens_INV.xlsx'.format(year, str(week).zfill(2)), index=False)